In [1]:
import time
import requests
import pandas as pd
from dotenv import dotenv_values

In [2]:
# constants 
total_transactions = 500 
collection_orderbook = 'CvRLWnb4Xn5dksNgi8t2vsdPtTE7JUhnWVnbdbGuyFNX'  
limit_transactions = 1000
limit_balances = 5
url = dotenv_values('.env')['private-rpc']
headers = {"Content-Type": "application/json"}
quick_break = 0.5

In [3]:
# transactions of orderbook
signature_list = []
for x in range(0, total_transactions, limit_transactions):
    if x+limit_transactions > total_transactions:
        if x == 0:
            payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": total_transactions-x }]}
        else:
            payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": total_transactions-x, "before": signature_list[-1] }]}   
    else:
        if x == 0:
            payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": limit_transactions }]}
        else:
            payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": limit_transactions, "before": signature_list[-1] }]}    
    r = requests.post(url, json=payload, headers=headers)
    if r.status_code != 200: 
        print(r.status_code)
        break     
    data = r.json()
    for x in data['result']:
        if x['confirmationStatus'] != 'processed':
            signature_list.append(x['signature'])
    time.sleep(quick_break)

In [4]:
# offered loans 
blocktime_list = []
user_list = []
escrow_list = []
check_list = []
for i in signature_list:
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getTransaction", "params": [i, {"encoding":"json", "maxSupportedTransactionVersion":0}]}
    r = requests.post(url, json=payload, headers=headers)
    if r.status_code != 200: 
        print(r.status_code)
        break 
    data = r.json()
    if list(data.keys())[1] == 'result':
        count = 0
        rescind = False
        for x in data['result']['meta']['logMessages']:
            if 'OfferLoan' in x:
                count += 1 
            if 'RescindLoan' in x:
                rescind = True    
        blocktime = data['result']['blockTime'] 
        accounts = data['result']['transaction']['message']['accountKeys'] 
        instructions = data['result']['transaction']['message']['instructions']
        check_list.append(i)
        if count == 1 and not rescind:
            blocktime_list.append(blocktime)
            user_list.append(accounts[0]) 
            escrow_list.append(accounts[instructions[2]['accounts'][4]])   
        if count == 1 and rescind:
            blocktime_list.append(blocktime)
            user_list.append(accounts[0]) 
            escrow_list.append(accounts[instructions[1]['accounts'][4]])              
        if count == 3 and not rescind:
            blocktime_list.extend([blocktime, blocktime, blocktime])
            user_list.extend([accounts[0], accounts[0], accounts[0]]) 
            escrow_list.extend([accounts[instructions[2]['accounts'][4]], accounts[instructions[3]['accounts'][4]], accounts[instructions[4]['accounts'][4]]])      
        if count == 9 and not rescind:
            print(i)
    else:
        print(data['error']['code'], data['error']['message'])   
        break       
    time.sleep(quick_break)

In [5]:
# offered amount of loans 
amount_list = []
for x in range(0, len(escrow_list), limit_balances):
    chunk = escrow_list[x:x+limit_balances]
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getMultipleAccounts", "params": [chunk,{"encoding": "base58"}]}
    r = requests.post(url, json=payload, headers=headers)
    if r.status_code != 200: 
        print(r.status_code)
        break      
    data = r.json()
    for x in data['result']['value']:
        if x is not None:
            amount_list.append(x['lamports'])
        else:
            amount_list.append(0)                 
    time.sleep(quick_break)

In [6]:
# top 20 loan offers
offers = pd.DataFrame({'blocktime': blocktime_list, 'amount': amount_list, 'user': user_list, 'escrow': escrow_list,})
offers['amount'] *= 0.000000001
offers['amount'] = round(offers['amount'], 2)
offers['user_readable'] = offers['user'].str[0:3] + '___' + offers['user'].str[-3:]
offers['datetime'] = pd.to_datetime(offers['blocktime'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
offers_desc = offers[offers['amount'] > 0.01][['amount', 'user_readable', 'datetime']].sort_values(['amount', 'datetime'], ascending=[False, True]).reset_index(drop=True)
offers_desc.index += 1
offers_desc.head(20)

,amount,user_readable,datetime
1,0.70,kF8___4xj,2024-10-30 21:26:34
2,0.69,9Lm___VLQ,2024-10-30 20:25:12
3,0.69,9Lm___VLQ,2024-10-30 20:25:12
4,0.69,9Lm___VLQ,2024-10-30 20:25:12
5,0.67,5kb___WXM,2024-10-30 17:58:13
6,0.67,GYe___Jzn,2024-10-30 18:01:44
7,0.66,Ekc___TK6,2024-10-30 17:10:00
8,0.66,Ekc___TK6,2024-10-30 17:10:00
9,0.66,Ekc___TK6,2024-10-30 17:10:00
10,0.66,Ekc___TK6,2024-10-30 17:10:37


In [7]:
# last updated 2024/10/30
r.status_code

200